In [1]:
import argparse
import uproot
import glob
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
tree = "Delphes"
branches = ["Track.PID", "Track.PT", "Track.P", "Track.Eta", "Track.Phi", "Track.EtaOuter", "Track.PhiOuter", "Track.XOuter", "Track.YOuter"]
pionPID = 211 # plus/minus

In [3]:
def getDataFromRootFile(file):
    with uproot.open(file) as f:
        # load the branches
        temp = {}
        for branch in branches:
            temp[branch] = np.array(ak.flatten(f[tree][branch].array()))
        
        # selection
        cut = (abs(temp["Track.PID"])==pionPID)

        # apply selection
        for branch in branches:
            temp[branch] = temp[branch][cut]
        
        # Original Method:
        #   phi = alpha - pi
        #   theta = beta - pi
        beta1 = temp["Track.PhiOuter"]+np.pi

        # Randomly generate beta between minimum and maximum allowed values
        betamin = 1.31272
        betamax = 1.82887
        beta2 = np.random.uniform(betamin, betamax,size=len(temp["Track.PID"]))
        
        # Determine yentry and ylocal based on beta
        yentry = 30/np.tan(beta2)
        ylocal=yentry+0.08125

        # Get angle of hit location with respect to barrel
        gamma0 = np.arctan2(temp["Track.YOuter"],temp["Track.XOuter"])

        # Get angle of sensor center with respect to barrel
        gamma=gamma0+yentry/30

        # Define unit vector of track at tracker edge with respect to barrel
        theta=2*np.arctan(np.exp(-temp["Track.EtaOuter"]))
        phi=temp["Track.PhiOuter"]
        x=np.sin(theta)*np.cos(phi)
        y=np.sin(theta)*np.sin(phi)
        z=np.cos(theta)

        # Transform into rotated coordinate system (sensor coordinate system sort of)
        xp=x*np.cos(gamma)+y*np.sin(gamma)
        yp=-x*np.sin(gamma)+y*np.cos(gamma)

        beta3=np.arctan2(yp,xp)+np.pi/2
        alpha=np.arctan2(np.sqrt(xp**2+yp**2),z)
        
    df = pd.DataFrame()
    df["Eta"] = temp["Track.Eta"]
    df["EtaOuter"] = temp["Track.EtaOuter"]
    df["Phi"] = temp["Track.Phi"]
    df["PhiOuter"] = temp["Track.PhiOuter"]
    df["Beta1"] = beta1
    df["Beta2"] = beta2
    df["Beta3"] = beta3
    df["Gamma"] = gamma
    df["alpha"]=alpha
    
    return df

In [4]:
files = "/home/elizahoward/cmspix28-mc-sim/temp/"
file = files+"minbias_0.00_0.10_GeV.root"

In [5]:
df = getDataFromRootFile(file)


In [6]:
df

,Eta,EtaOuter,Phi,PhiOuter,Beta1,Beta2,Beta3,Gamma,alpha
0,-0.093881,-0.093898,2.287534,2.320269,5.461862,1.444017,1.443333,2.447732,1.664556
1,2.143140,2.143220,3.017493,3.039766,6.181358,1.423662,1.422591,3.187971,0.233487
2,-0.706421,-0.706534,1.405576,1.372224,4.513816,1.635131,1.635219,1.307801,2.224964
3,-2.540730,-2.540777,-2.237474,-2.220539,0.921053,1.541152,1.541144,-2.190887,2.984308
4,1.430900,1.431146,-0.738293,-0.778899,2.362694,1.682949,1.683422,-0.891524,0.469265
...,...,...,...,...,...,...,...,...,...
2057,-1.283975,-1.284078,-2.293600,-2.320383,0.821209,1.354986,1.351571,-2.101158,2.601318
2058,0.075657,0.075741,2.012954,2.095078,5.236671,1.785040,1.788379,1.877496,1.495127
2059,-1.050694,-1.050812,-2.443119,-2.473257,0.668336,1.398920,1.397208,-2.299668,2.468860
2060,-0.395777,-0.395783,-2.834253,-2.844519,0.297074,1.515048,1.514991,-2.788714,1.956634


In [24]:
np.max(df.Beta3)

3.1404004122122204

In [25]:
np.min(df.Beta3)

-3.1414332489236867